In [1]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import glob
import pandas as pd
import os
import glob

# Personnal Import 
from utils.utilities import get_mode_date2path
from build_inputs.load_raw_data import load_subway_15_min,load_CRITER,load_netmob

# Data
- Validation individuelles, aggrégée **3min**
- Metro (entrée/sortie)  **15 min**
- Sensor, aggrégé **6 min**

### Notes et questionnements
- Un 'VAL_ARRET_CODE' peut être l'arrêt de plusieurs mêmes bus, voir d'un même bus et d'un même arrêt de métro. Où d'un même bus et d'un même arrêt de tram. Je dois donc nommer différement les VAL_ARRET_CODE de chacun des modes. Une proposition est de mettre le mode (B,S,T) devant les id. Comme ça on pourra regrouper sans soucis.
- La moyenne des déplacement de la df_subway est de 5 trajet toute les 3 minutes, quelque soit la station et l'heure (d'ouverture) considéré. Max 88.
- Ok on a aggrégé 3 min, mais est-ce qu'on peut recouper les sorties 3min avec les validation + Sortie de métro 15 min? 

In [2]:
# Subway 15 Min :                                   [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,~, , ,~,~, , ],[J,F,M, , , , , , , , , ]
# Validation Individuelles  Nov 2019 - Mai 2021     [ , , , , , , , , , ,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M, , , , , , , ]
# NetMob 15 Min :                                   [ , ,M,A,M, , , , , , , ],[ , , , , , , , , , , , ],[ , , , , , , , , , , , ]
# Sensor 6 Min :  Janvier 2019 - Decembre 2020      [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,J,J,A,S,O,N,D]

## Load Validation Individuelles
Load 3 df : df_sub, df_tram, df_bus

In [3]:
FOLDER_PATH = '../../../data/rrochas/raw_data/keolis_data_2019-2020/'
valid_ind_path = f"{FOLDER_PATH}Sub_Tram_11_2019_03_2020"
dates = ['11-2019','12-2019','1-2020','2-2020','3-2020']

subway_paths, tramway_paths, bus_paths = sorted(glob.glob(os.path.join(valid_ind_path, "*df_subway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_tramway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_bus*.csv")))
mode_month2path = get_mode_date2path([subway_paths,tramway_paths,bus_paths],['sub','tram','bus'])

for name in ['sub','tram','bus']:
    globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])

/tmp/ipykernel_108571/2947046094.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/tmp/ipykernel_108571/2947046094.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/tmp/ipykernel_108571/2947046094.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/tmp/ipykernel_108571/2947046094.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/tmp/ipykern

In [ ]:
from datetime import datetime 
from load_inputs.subway_in import get_trigram_correspondance

start,end = datetime(2019,11,1),datetime(2020,4,1)
freqs = ['2min','3min','5min','6min','10min','15min','30min','1H']
freqs.reverse()

save_path = '../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle'

for name in ['sub','tram','bus']: # ['sub','tram','bus']
    df_i = globals()[f'df_{name}']
    df_i.VAL_DATE = pd.to_datetime(df_i.VAL_DATE)
    for freq in freqs:
        reindex = pd.date_range(start,end,freq = freq)
        if name == 'sub':
            name_local_folder = 'subway_indiv_'
            df = df_i[['Flow','VAL_DATE','CRS_SENS_TRAJET','LIB_STA_SIFO']].groupby(['LIB_STA_SIFO',pd.Grouper(key='VAL_DATE',freq=freq)]
                                                                                    ).agg(Flow = ('Flow','sum')).reset_index()
            df = df.pivot(index = 'VAL_DATE',columns='LIB_STA_SIFO',values ='Flow').fillna(0)
            df_corres = get_trigram_correspondance()
            df = df.rename(columns = {row.INDIV:row.COD_TRG for k,row in df_corres.iterrows()})
        elif name == 'tram':
            name_local_folder = 'tramway_indiv_'
            df = df_i.groupby(['LIG_NUMERO_SAE','VAL_ARRET_CODE','CRS_SENS_TRAJET','COD_LIG_CLI',pd.Grouper(key='VAL_DATE',freq=freq)]
                              ).agg(Flow = ('Flow','sum')).reset_index()
        elif name == 'bus':
            name_local_folder = 'bus_indiv_'
            df_i.COD_LIG_CLI = df_i.COD_LIG_CLI.astype(str)
            df = df_i.groupby(['LIG_NUMERO_SAE','VAL_ARRET_CODE','CRS_SENS_TRAJET','COD_LIG_CLI',pd.Grouper(key='VAL_DATE',freq=freq)]
                              ).agg(Flow = ('Flow','sum')).reset_index()
        else:
            raise NotImplementedError(f"Mode {name} not implemented yet")
        
        # save
        if False:
            dir_path = f"{save_path}/{name_local_folder}{freq}"
            if not os.path.exists(dir_path):
                os.makedirs(dir_path)
            df.to_csv(f"{dir_path}/{name_local_folder}{freq}.csv")

/tmp/ipykernel_108571/2536032855.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  reindex = pd.date_range(start,end,freq = freq)
/tmp/ipykernel_108571/2536032855.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df_i[['Flow','VAL_DATE','CRS_SENS_TRAJET','LIB_STA_SIFO']].groupby(['LIB_STA_SIFO',pd.Grouper(key='VAL_DATE',freq=freq)]
/tmp/ipykernel_108571/2536032855.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  reindex = pd.date_range(start,end,freq = freq)
/tmp/ipykernel_108571/2536032855.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df_i.groupby(['LIG_NUMERO_SAE','VAL_ARRET_CODE','CRS_SENS_TRAJET','COD_LIG_CLI',pd.Grouper(key='VAL_DATE',freq=freq)]
/tmp/ipykernel_108571/2536032855.py:14: FutureWarning: 'H' is deprecated and will be removed in a

In [41]:
for name_local_folder in ['subway_indiv_','tramway_indiv_','bus_indiv_']:
    freq = '15min'
    print(save_path)
    print(name_local_folder)
    print(freq)
    display(pd.read_csv(f"{save_path}/{name_local_folder}{freq}/{name_local_folder}{freq}.csv").head())

../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle
subway_indiv_
15min


,VAL_DATE,AMP,BEL,BRO,CHA,COR,CPA,CRO,CUI,CUS,...,PER,GUI,JAU,REP,SAN,SAX,GER,VMY,SOI,JEA
0,2019-11-01 00:00:00,20.0,164.0,10.0,59.0,88.0,5.0,20.0,1.0,5.0,...,89.0,13.0,16.0,12.0,49.0,68.0,8.0,23.0,3.0,59.0
1,2019-11-01 00:15:00,6.0,82.0,11.0,30.0,43.0,3.0,8.0,0.0,2.0,...,19.0,7.0,7.0,3.0,11.0,33.0,2.0,16.0,0.0,38.0
2,2019-11-01 00:30:00,0.0,4.0,0.0,0.0,2.0,2.0,6.0,0.0,1.0,...,0.0,0.0,0.0,3.0,5.0,2.0,0.0,3.0,1.0,0.0
3,2019-11-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-11-01 04:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0


../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle
tramway_indiv_
15min


,Unnamed: 0,LIG_NUMERO_SAE,VAL_ARRET_CODE,CRS_SENS_TRAJET,COD_LIG_CLI,VAL_DATE,Flow
0,0,520,32102,0,T1,2019-11-02 15:00:00,66
1,1,520,32102,0,T1,2019-11-02 17:30:00,53
2,2,520,32102,0,T1,2019-11-02 17:45:00,72
3,3,520,32102,0,T1,2019-11-02 18:30:00,26
4,4,520,32102,0,T1,2019-11-03 04:45:00,1


../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle
bus_indiv_
15min


/tmp/ipykernel_108571/2839408299.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  display(pd.read_csv(f"{save_path}/{name_local_folder}{freq}/{name_local_folder}{freq}.csv").head())


,Unnamed: 0,LIG_NUMERO_SAE,VAL_ARRET_CODE,CRS_SENS_TRAJET,COD_LIG_CLI,VAL_DATE,Flow
0,0,2,202,1,2,2019-11-01 12:15:00,1
1,1,2,202,1,2,2019-11-02 10:30:00,1
2,2,2,202,1,2,2019-11-02 12:00:00,1
3,3,2,202,1,2,2019-11-04 08:00:00,1
4,4,2,202,1,2,2019-11-04 08:15:00,1


## Load Subway 15 min

In [ ]:
FOLDER_PATH = '../../../data/rrochas/raw_data/keolis_data_2019-2020/'

txt_name = "Métro 15 minutes 2019 2020.txt"  # 1 january 2019 -  22 octobre 2020
df_metro_funi_2019_2020 = load_subway_15_min(f"{FOLDER_PATH}{txt_name}")
display(df_metro_funi_2019_2020.head(3))

txt_name = "Métro 15 minutes 2021.txt" # 1 january 2021 -  31 march 2021
df_metro_funi_2021 = load_subway_15_min(f"{FOLDER_PATH}{txt_name}")
display(df_metro_funi_2021.head(3))

subway_in = df_metro_funi_2019_2020.drop(columns = ['out'])
subway_in = subway_in.pivot_table(index = ['datetime'],columns = 'Station',values = 'in').fillna(0)

subway_out = df_metro_funi_2019_2020.drop(columns = ['in'])
subway_out = subway_out.pivot_table(index = ['datetime'],columns = 'Station',values = 'out').fillna(0)

display(subway_in.head(3))
display(subway_out.head(3))

# save: 
if False : 
    dirname = '../../../data/rrochas/prediction_validation'
    subway_in.to_csv(f"{dirname}/subway_in/subway_in_all.csv")
    subway_out.to_csv(f"{dirname}/subway_out/subway_out_all.csv")

,datetime,Station,Code ligne,in,out
0,2019-01-01 00:00:00,Ampère Victor Hugo,A,2,4.0
1,2019-01-01 00:15:00,Ampère Victor Hugo,A,3,2.0
2,2019-01-01 00:30:00,Ampère Victor Hugo,A,3,7.0


,datetime,Station,Code ligne,in,out
0,2021-03-01 05:00:00,Hôtel de ville - Louis Pradel,A,13,12
1,2021-03-01 05:45:00,Hôtel de ville - Louis Pradel,A,29,22
2,2021-03-01 06:45:00,Hôtel de ville - Louis Pradel,A,62,54


Station,Ampère Victor Hugo,Bellecour,Brotteaux,Charpennes,Cordeliers,Croix Paquet,Croix-Rousse,Cuire,Cusset,Debourg,...,Perrache,Place Guichard,Place Jean Jaurès,République Villeurbanne,Sans Souci,Saxe - Gambetta,St Just,Stade de Gerland,Valmy,Vieux Lyon
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,2.0,26.5,6.0,10.5,21.0,4.0,1.0,2.0,3.0,3.0,...,38.0,2.0,8.0,6.0,6.0,11.5,0.0,7.0,7.0,5.666667
2019-01-01 00:15:00,3.0,14.5,2.0,7.0,8.0,0.0,3.0,2.0,1.0,5.0,...,25.0,3.0,6.0,5.0,3.0,19.0,0.0,9.0,8.0,6.333333
2019-01-01 00:30:00,3.0,9.5,3.0,9.5,20.0,1.0,2.0,0.0,1.0,6.0,...,15.0,3.0,6.0,1.0,6.0,9.5,0.0,0.0,3.0,3.666667


Station,Ampère Victor Hugo,Bellecour,Brotteaux,Charpennes,Cordeliers,Croix Paquet,Croix-Rousse,Cuire,Cusset,Debourg,...,Perrache,Place Guichard,Place Jean Jaurès,République Villeurbanne,Sans Souci,Saxe - Gambetta,St Just,Stade de Gerland,Valmy,Vieux Lyon
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,4.0,25.0,2.0,36.0,8.0,0.0,12.0,9.0,9.0,11.0,...,16.0,12.0,12.0,16.0,18.0,8.5,0.0,12.0,25.0,4.666667
2019-01-01 00:15:00,2.0,14.0,3.0,411.0,4.0,0.0,8.0,3.0,21.0,15.0,...,37.0,6.0,15.0,10.0,22.0,23.0,0.0,7.0,11.0,1.666667
2019-01-01 00:30:00,7.0,18.0,2.0,209.5,10.0,4.0,8.0,7.0,13.0,8.0,...,6.0,8.0,17.0,7.0,14.0,22.0,0.0,3.0,25.0,3.666667


## Weird differences between data:
- Tout se passe comme si 'subway_in_algo' (utilisé jusqu'a maintenant) était quasi identique à 'subway_in_new', sauf sur 4 stations. 
- Sur ces quatres stations, il y a un facteur ~2.1 fois plus grand. 

In [ ]:
from load_inputs.subway_in import get_trigram_correspondance
df_correspondance = get_trigram_correspondance()

subway_in_new = subway_in.iloc[:31036]
subway_in_new = subway_in_new.reindex(pd.date_range(start = '2019-01-01',end = '2020-01-01',freq = '15min'),fill_value = 0)
subway_in_new.head(3)


dirname = '../../../data/rrochas/prediction_validation'
subway_in_algo = pd.read_csv(f'{dirname}/subway_in/subway_in.csv',index_col = 0)
subway_in_algo.index = pd.to_datetime(subway_in_algo.index)
subway_in_algo = subway_in_algo.rename(columns = {row.Station: row.COD_TRG for _,row in df_correspondance.iterrows()})
subway_in_algo = subway_in_algo[df_correspondance.COD_TRG]
subway_in_algo.head(3)

from plotting.TS_analysis import plot_TS
df_diff = subway_in_algo[:-1] - subway_in_new[:-1]
mask = df_diff.quantile(0.99) > 1e-5
plot_TS(df_diff[mask[mask].index.to_list()][:2000],width = 1200,height = 400,bool_show=True,title= 'difference 2 datasets')

,AMP,BEL,BRO,CHA,COR,CPA,CRO,CUI,CUS,DEB,...,PAR,PER,GUI,JAU,REP,SAN,SAX,GER,VMY,JEA
2019-01-01 00:00:00,2.0,53.0,6.0,21.0,21.0,4.0,1.0,2.0,3.0,3.0,...,74.0,38.0,2.0,8.0,6.0,6.0,23.0,7.0,7.0,12.0
2019-01-01 00:15:00,3.0,29.0,2.0,14.0,8.0,0.0,3.0,2.0,1.0,5.0,...,105.0,25.0,3.0,6.0,5.0,3.0,38.0,9.0,8.0,13.0
2019-01-01 00:30:00,3.0,19.0,3.0,19.0,20.0,1.0,2.0,0.0,1.0,6.0,...,18.0,15.0,3.0,6.0,1.0,6.0,19.0,0.0,3.0,8.0


## Différence avec les validation individuelles: 

In [128]:
df_in_indiv = pd.read_csv(f"{dirname}/agg_data/validation_individuelle/subway_indiv_15min/subway_indiv_15min.csv",index_col = 0)
df_in_indiv

,AMP,BEL,BRO,CHA,COR,CPA,CRO,CUI,CUS,DEB,...,PER,GUI,JAU,REP,SAN,SAX,GER,VMY,SOI,JEA
VAL_DATE,,,,,,,,,,,,,,,,,,,,,
2019-11-01 00:00:00,20.0,164.0,10.0,59.0,88.0,5.0,20.0,1.0,5.0,21.0,...,89.0,13.0,16.0,12.0,49.0,68.0,8.0,23.0,3.0,59.0
2019-11-01 00:15:00,6.0,82.0,11.0,30.0,43.0,3.0,8.0,0.0,2.0,3.0,...,19.0,7.0,7.0,3.0,11.0,33.0,2.0,16.0,0.0,38.0
2019-11-01 00:30:00,0.0,4.0,0.0,0.0,2.0,2.0,6.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,5.0,2.0,0.0,3.0,1.0,0.0
2019-11-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-11-01 04:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-30 22:15:00,0.0,5.0,2.0,8.0,4.0,0.0,2.0,2.0,4.0,1.0,...,4.0,2.0,0.0,1.0,0.0,9.0,0.0,3.0,8.0,4.0
2020-03-30 22:30:00,1.0,5.0,0.0,13.0,0.0,0.0,2.0,1.0,2.0,0.0,...,3.0,2.0,1.0,1.0,2.0,1.0,5.0,5.0,7.0,0.0
2020-03-30 22:45:00,1.0,4.0,0.0,11.0,1.0,0.0,2.0,0.0,4.0,4.0,...,3.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,3.0,1.0


In [ ]:
df2plot.index = pd.to_datetime(df2plot.index)

,BEL,BLA,BON,CHA,GIL,HOT,JEA,MAC,PAR,PER,SAX,SOI,VAI,VEN
VAL_DATE,,,,,,,,,,,,,,
2019-11-01 00:00:00,41.5,-24.0,-2.0,22.0,6.0,50.5,30.333333,-15.0,19.0,-51.0,26.0,-39.0,-30.0,-3.0
2019-11-01 00:15:00,-17.0,8.0,-3.0,-6.5,-58.0,-23.5,15.333333,-28.0,-63.0,-96.0,-13.0,-4.0,-6.0,-10.0
2019-11-01 00:30:00,-51.0,-22.0,-1.0,-22.0,-57.0,-66.5,-15.000000,-22.0,-13.0,-36.0,-25.0,1.0,-18.0,-8.0
2019-11-01 00:45:00,-4.5,-9.0,0.0,-5.5,-7.0,0.0,0.000000,0.0,-4.0,-1.0,-1.0,-3.0,-7.0,-4.0
2019-11-01 04:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-30 22:15:00,-0.5,6.0,4.0,4.0,-15.0,4.5,3.666667,-3.0,-13.0,4.0,5.0,4.0,-2.0,-2.0
2020-03-30 22:30:00,1.0,-1.0,4.0,12.5,3.0,3.0,-1.666667,-1.0,2.0,0.0,-0.5,5.0,-4.0,-1.0
2020-03-30 22:45:00,0.5,-3.0,2.0,11.0,2.0,6.0,1.000000,1.0,2.0,3.0,2.0,3.0,3.0,3.0


In [148]:
subway_in_2020 = subway_in.reindex(df_in_indiv.index).fillna(0).copy()

diff_2020 = (df_in_indiv - subway_in_2020)
mask = abs(diff_2020.mean()) > 10
df2plot = diff_2020[mask[mask].index.to_list()]
df2plot.index = pd.to_datetime(df2plot.index)
plot_TS(df2plot.iloc[:2000],width = 1200,height = 400,bool_show=True,title= 'difference 2 datasets')

Loading BokehJS ...

figure(id='p8065', ...)

In [ ]:
dirname = '../../../data/rrochas/prediction_validation'
print(os.listdir(f"{dirname}"))
os.listdir(f"{dirname}/agg_data/validation_individuelle/subway_indiv_15min")

['METR_LA', 'PEMS_BAY', 'PEMS_d3', 'PEMS_d4', 'PEMS_d7', 'data_bidon', 'generate_data.ipynb', 'netmob_bidon', 'ref_subway.csv', 'subway_in', 'netmob_video_lyon', 'netmob_image_per_station', 'NetMob_lyon.geojson', 'lyon_iris_shapefile', 'POIs', 'subway_out', 'CRITER_3lanes', 'agg_data']


['subway_indiv_3min',
 'tramway_indiv_3min',
 'bus_indiv_3min',
 'vaf_indiv_3min',
 'subway_indiv_5min',
 'tramway_indiv_5min',
 'bus_indiv_5min',
 'vaf_indiv_5min',
 'subway_indiv_6min',
 'tramway_indiv_6min',
 'bus_indiv_6min',
 'vaf_indiv_6min',
 'subway_indiv_10min',
 'tramway_indiv_10min',
 'bus_indiv_10min',
 'vaf_indiv_10min',
 'subway_indiv_15min',
 'tramway_indiv_15min',
 'bus_indiv_15min',
 'vaf_indiv_15min',
 'subway_indiv_1H',
 'subway_indiv_30min',
 'subway_indiv_2min',
 'tramway_indiv_1H',
 'tramway_indiv_30min',
 'tramway_indiv_2min',
 'bus_indiv_1H',
 'bus_indiv_30min',
 'bus_indiv_2min']

## Load Velo'V 

In [ ]:
import os 
import sys 
import pandas as pd
import chardet
import json 
from shapely.geometry import Point
import geopandas as gpd


def get_velov_data(year,quarter,folder_path):
    ''' Return shared bike dataframe from "Lyon Velov"  
    'year' in [2019,2020]
    'quarter' in ['T1','T2','T3','T4']
    '''
    # usefull to get the "encoding" of the csv, so that "pd.read_csv" can work well
    pathway = f'{folder_path}/Trajets VELOV {year}/VELOV_TRAJETS_{str(year)}_{quarter}.csv'
    with open(pathway, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(100000))      
    # ...
    
    df_velov = pd.read_csv(pathway, encoding = result['encoding'], sep = ";")
    df_velov.columns=['id_sortie','nom_sortie','date_sortie','id_retour','nom_retour','date_retour']
    df_velov.date_sortie = pd.to_datetime(df_velov.date_sortie)
    df_velov.date_retour = pd.to_datetime(df_velov.date_retour)
    return(df_velov)



FOLDER_PATH = '../../../data/rrochas/raw_data/VELO-LYON'
df = pd.concat([get_velov_data(year,Ti,FOLDER_PATH) for year in [2019,2020] for Ti in ['T1','T2','T3','T4']]) 
print('length df: ',len(df))
df.head()


length df:  15586007


,id_sortie,nom_sortie,date_sortie,id_retour,nom_retour,date_retour
0,6005,6005 - PLACE EDGAR QUINET,2019-01-01 00:00:00,2024.0,2024 - RÉPUBLIQUE / MAUPIN,2019-01-01 00:20:00
1,10072,10072 - JACQUES BREL,2019-01-01 00:01:00,4012.0,4012 - PLACE ADRIEN GODIEN,2019-01-01 00:09:00
2,6037,6037 - CITÉ INTERNATIONALE / INTERPOL,2019-01-01 00:02:00,10025.0,10025 - TOTEM,2019-01-01 00:32:00
3,3012,3012 - PLACE DU CHÂTEAU,2019-01-01 00:06:00,7002.0,7002 - UNIVERSITÉS LYON III / LYON II,2019-01-01 00:26:00
4,1002,1002 - OPÉRA,2019-01-01 00:07:00,7056.0,7056 - PLACE RASPAIL,2019-01-01 00:22:00


In [ ]:
html_save_path = 'velov_map.html'
json_path = 'locations/pvo_patrimoine_voirie.pvostationvelov.json'
stations_json = json.load(open(f"{FOLDER_PATH}/{json_path}",'rb'))

gdf = gpd.GeoDataFrame(
    [
        {**feat['properties'],
         'geometry': Point(*feat['geometry']['coordinates'])}
        for feat in stations_json['features']
    ],
    crs="EPSG:4326"
)
gdf.head()

In [ ]:
# save:
if False:
    save_path = '../../../../data/rrochas/prediction_validation/agg_data/velov'
    for temp_agg in ['2min','3min','5min','10min','15min','30min','1H']:
        df_emitted = df[['id_sortie','date_sortie','id_retour']].groupby(['id_sortie',pd.Grouper(freq=temp_agg,key='date_sortie')]).agg(volume = ('id_retour','count'))
        df_attracted = df[['id_sortie','date_retour','id_retour']].groupby(['id_retour',pd.Grouper(freq=temp_agg,key='date_retour')]).agg(volume = ('id_sortie','count'))
        df_emitted.to_csv(f"{save_path}/velov_emitted_by_station{temp_agg}.csv")
        df_attracted.to_csv(f"{save_path}/velov_attracted_by_station{temp_agg}.csv")

## Load NetMob 15 min

In [ ]:
FOLDER_PATH = '../../Data/NetMob/'
apps = [app for app in os.listdir(FOLDER_PATH) if ((app != 'Lyon.geojson') and (not app.startswith('.'))) ]   # Avoid hidden folder and Lyon.geojson

In [5]:
app = apps[0]
folder_days = [day for day in os.listdir(f'{FOLDER_PATH}/{app}') if (not day.startswith('.')) ] 
day = folder_days[0]
txt_paths = sorted(glob.glob(os.path.join(f'{FOLDER_PATH}/{app}/{day}', "*.txt")))
txt_paths

['../../Data/NetMob//Apple_Video/20190504/Lyon_Apple_Video_20190504_DL.txt',
 '../../Data/NetMob//Apple_Video/20190504/Lyon_Apple_Video_20190504_UL.txt']

In [6]:
txt_path = txt_paths[0]
df_Apple_Vid_DL = load_netmob(txt_path,day)
df_Apple_Vid_DL.head()

,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
tile_id,,,,,,,,,,,,,,,,,,,,,
66,1834,1596,1378,1575,637,421,203,1366,547,254,...,4114,2992,2887,3297,4634,4208,4886,2095,2066,919
353,2000,1687,1357,1548,764,484,195,2492,591,407,...,4673,3288,3156,3778,5123,4400,5300,2187,2176,984
354,1621,1540,1258,1381,490,375,94,1131,453,302,...,3887,3128,3048,3337,4582,3792,4519,2189,2309,984
640,1803,1493,1201,1332,632,514,73,2951,428,370,...,4404,3352,3082,3512,4697,4056,4862,2155,2156,1074
641,1621,1583,1296,1383,493,415,75,1087,402,368,...,4064,3426,3206,3517,4834,4011,4719,2256,2230,1073


## Load Sensor 6 min

In [ ]:
years = [2019,2020]
#FOLDER_PATH = '../../Data/Comptages_Velo_Routier/CRITER/'
FOLDER_PATH = '../../../data/rrochas/raw_data/Comptages_Velo_Routier/CRITER/'
list_files = sorted(glob.glob(os.path.join(f'{FOLDER_PATH}6 min {years[0]}', "*.txt"))) + sorted(glob.glob(os.path.join(f'{FOLDER_PATH}6 min {years[1]}', "*.txt")))
oct_2019_mar_2020 = list_files[9:15]

In [ ]:
df = load_CRITER(oct_2019_mar_2020[0])
len_init = len(df)
print((list_files[0]))
df.head()

/home/rrochas/prediction-validation/build_inputs/load_raw_data.py:48: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(txt_path, sep=';',infer_datetime_format=True)


../../../data/rrochas/raw_data/Comptages_Velo_Routier/CRITER/6 min 2019/6mn_01_Janvier_2019.txt


,ID_POINT_MESURE,NOM_POINT_MESURE,DEBIT_HEURE,TAUX_HEURE,HORODATE,NOMBRE_ECH_1_MIN_MANQUANTS,day,str_hour_min,hour_min
0,1,Berthelot_Servant_Mace,380,3,2019-10-01 00:00:00,0,1,0:0,0.0
1,1,Berthelot_Servant_Mace,240,2,2019-10-01 00:06:00,0,1,0:6,1.0
2,1,Berthelot_Servant_Mace,270,2,2019-10-01 00:12:00,0,1,0:12,2.0
3,1,Berthelot_Servant_Mace,270,2,2019-10-01 00:18:00,0,1,0:18,3.0
4,1,Berthelot_Servant_Mace,200,1,2019-10-01 00:24:00,0,1,0:24,4.0


# 1er Etape : Prédiction Métro
- On va d'abord prédire la demande sur une ligne (disons A).  
- On va comparer des modèle : LSTM, CNN, CNN-LSTM, GNN.
    - A priori pas de "raison" que le GNN marche mieux. Si c'est le cas, c'est peut être simplement que le modèle est plus complexe, mais j'ai du mal à croire que si on donne les bonnes informations (historique -7d, -1d, -4,3,2,1t), on a des meilleurs résultats avec GNN. Sauf si il y a des relation asynchrone "récurrentes", mais sans causalité. De la même manière que l'historique -7d sert de référence, mais ne témoigne pas d'un lien causal.
- Identifier des moments interessants : anomalies sur entrée/sortie métro. Voir les prédictions sur ces moments là particulier.
